![NCIA](NCIA_Images\start.png)

# Building Neural Networks with PyTorch

### Setup

Este projeto requer Python 3.10 ou superior:

In [ ]:
!pip install -q pandas matplotlib seaborn scikit-learn

You should consider upgrading via the 'C:\Users\antonio.fontenele\Documents\NCIA\Material_Aulas\Aula 32\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


<div align="center" style="margin-top:20px; margin-bottom:20px;">
  
![pt](Aula_Imagens/pytorch.png)

</div>


Isso demora 9 minutos =/

In [ ]:
!pip install torch>=2.8.0

You should consider upgrading via the 'C:\Users\antonio.fontenele\Documents\NCIA\Material_Aulas\Aula 32\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


In [ ]:
import sys
assert sys.version_info >= (3, 10)

Também requer Scikit-Learn ≥ 1.6.1:

In [ ]:
from packaging.version import Version
import sklearn
assert Version(sklearn.__version__) >= Version("1.6.1")

E é claro que precisamos do PyTorch, especificamente do PyTorch ≥ 2.8.0:

In [ ]:
import torch

assert Version(torch.__version__) >= Version("2.8.0")

Como fizemos nos capítulos anteriores, vamos definir os tamanhos de fonte padrão para deixar as figuras mais bonitas:

In [ ]:
import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

## Sobre o Pytorch

PyTorch é uma biblioteca open source de deep learning (originada do Torch/Lua) mantida pelo laboratório FAIR (Meta AI). Em Python, oferece **tensores**, **autodiferenciação (autograd)**, **aceleração por GPU/TPU** e um ecossistema de **camadas, perdas e otimizadores prontos**, tornando o desenvolvimento de redes neurais ágil e produtivo. Conceitualmente, lembra o NumPy, mas com suporte nativo a **cálculo em hardware acelerado** e **gradientes automáticos**.

"PôfêSô, por que Pt? eu gostava do TF!" Em 2016, o **TensorFlow** dominava o cenário por desempenho e escalabilidade, porém seu modelo de programação era mais **estático e complexo**. O PyTorch foi criado com uma proposta **mais “pythônica” e dinâmica**, usando **grafos computacionais dinâmicos (define-by-run)**, o que facilita a **depuração**, a **exploração interativa** e a **pesquisa**.
Além do design limpo e documentação robusta, a comunidade open source cresceu rapidamente; em 2022, a governança migrou para a **PyTorch Foundation** (Linux Foundation), consolidando o ecossistema. O resultado prático foi a **adoção massiva na academia** e, por consequência, **migração gradual da indústria**.

## O que você aprenderá neste capítulo

* **Fundamentos**: como trabalhar com **tensores** e **autograd** no PyTorch.
* **Primeiros passos de modelagem**: construir e treinar um **modelo de regressão linear** para entender o pipeline básico.
* **Evolução para redes profundas**: ampliar para **redes multicamadas (MLP)**:

  * **Regressão** com redes neurais.
  * **Classificação** com redes neurais.
* **Arquiteturas personalizadas**: criar modelos com **múltiplas entradas** ou **múltiplas saídas**.
* **Ajuste de hiperparâmetros**: usar **Optuna** para **tunar** modelos automaticamente.
* **Otimização e exportação**: técnicas para **otimizar desempenho** e **salvar/exportar** modelos para uso em produção.

> **Resumo da ideia central**: PyTorch equilibra **simplicidade**, **flexibilidade** e **alto desempenho** graças aos **grafos dinâmicos** e ao **ecossistema maduro**. Este capítulo guia você do **básico (tensores/autograd)** à **produção (tuning, otimização e exportação)** passando por exemplos práticos de **regressão** e **classificação**.


# 1. Fundamentos do PyTorch

O **tensor** é a estrutura de dados central do PyTorch: um **array multidimensional** com **forma** e **tipo de dado**, usado para computação numérica. Ele é semelhante a um array do **NumPy**, mas tem duas vantagens fundamentais: pode **residir em GPU** (ou outros aceleradores) e **suporta auto-diferenciação**. A partir deste ponto, **todos os modelos** trabalharão **recebendo e produzindo tensores**, de modo análogo a como modelos do Scikit-Learn usam arrays NumPy. O próximo passo é **criar e manipular tensores**.

## 1.1 PyTorch Tensors

Você pode criar um tensor PyTorch da mesma forma que criaria um array NumPy. Por exemplo, vamos criar um array 2 × 3:

In [6]:
import torch

X = torch.tensor([[1.0, 4.0, 7.0], [2.0, 3.0, 6.0]])
X

tensor([[1., 4., 7.],
        [2., 3., 6.]])

Assim como um array do NumPy, um **tensor** pode conter **floats**, **inteiros**, **booleanos** ou **números complexos** — **apenas um tipo por tensor**. Se você o inicializa com valores de tipos mistos, o PyTorch escolhe o **mais geral** segundo a hierarquia: **complexo > float > inteiro > bool**. Também é possível **definir o tipo explicitamente** na criação (por exemplo, `dtype=torch.float16` para floats de 16 bits). **Tensores de strings ou objetos não são suportados**.

> Você pode **inspecionar a forma (shape) e o tipo (dtype)** de um tensor diretamente:


In [7]:
X.shape

torch.Size([2, 3])

In [8]:
X.dtype

torch.float32

A indexação funciona exatamente como para matrizes NumPy:

In [9]:
X[0, 1]

tensor(4.)

In [10]:
X[:, 1]

tensor([4., 3.])

Você pode aplicar **diversas operações numéricas** diretamente em tensores, com uma API **muito semelhante à do NumPy**: `torch.abs()`, `torch.cos()`, `torch.exp()`, `torch.max()`, `torch.mean()`, `torch.sqrt()`, entre outras.
Quase todas também existem como **métodos do próprio tensor**, permitindo escrever `X.exp()` em vez de `torch.exp(X)`. O próximo trecho demonstra algumas dessas operações na prática.

In [11]:
10 * (X + 1.0)  # item-wise addition and multiplication

tensor([[20., 50., 80.],
        [30., 40., 70.]])

In [12]:
X.exp()

tensor([[   2.7183,   54.5981, 1096.6332],
        [   7.3891,   20.0855,  403.4288]])

In [13]:
X.mean()

tensor(3.8333)

In [14]:
X.max(dim=0)

torch.return_types.max(
values=tensor([2., 4., 7.]),
indices=tensor([1, 0, 0]))

In [15]:
X @ X.T

tensor([[66., 56.],
        [56., 49.]])

Você também pode converter um tensor em uma matriz NumPy usando o método numpy() e criar um tensor a partir de uma matriz NumPy:

In [16]:
import numpy as np

X.numpy()

array([[1., 4., 7.],
       [2., 3., 6.]], dtype=float32)

In [17]:
torch.tensor(np.array([[1., 4., 7.], [2., 3., 6.]]))

tensor([[1., 4., 7.],
        [2., 3., 6.]], dtype=torch.float64)

No PyTorch, o **padrão de floats é 32 bits (float32)**, enquanto no NumPy é **64 bits (float64)**. Em deep learning, **float32 costuma ser preferível**: consome **metade da RAM**, **acelera os cálculos** e a rede **não precisa** da precisão extra de 64 bits.
Ao converter um array NumPy com `torch.tensor()`, **indique** `dtype=torch.float32`. Como alternativa, `torch.FloatTensor()` **já converte** automaticamente para **32 bits**.

In [18]:
torch.tensor(np.array([[1., 4., 7.], [2., 3., 6.]]), dtype=torch.float32)

tensor([[1., 4., 7.],
        [2., 3., 6.]])

In [19]:
torch.FloatTensor(np.array([[1., 4., 7.], [2., 3., 6]]))

tensor([[1., 4., 7.],
        [2., 3., 6.]])

In [20]:
# extra code: demonstrate torch.from_numpy()
X2_np = np.array([[1., 4., 7.], [2., 3., 6]])
X2 = torch.from_numpy(X2_np)  # X2_np and X2 share the same data in memory
X2_np[0, 1] = 88
X2

tensor([[ 1., 88.,  7.],
        [ 2.,  3.,  6.]], dtype=torch.float64)

Você também pode modificar um tensor no local usando indexação e fatiamento, como com uma matriz NumPy:

In [21]:
X[:, 1] = -99
X

tensor([[  1., -99.,   7.],
        [  2., -99.,   6.]])

A API do PyTorch oferece diversas operações **in-place** (terminadas com `_`), como `abs_()`, `sqrt_()` e `zero_()`, que **modificam o próprio tensor**. Elas podem **economizar memória** e **aumentar a velocidade** em alguns casos.
Exemplo: `relu_()` aplica a **ReLU** diretamente, **substituindo valores negativos por 0** no mesmo tensor.


In [22]:
X.relu_()
X

tensor([[1., 0., 7.],
        [2., 0., 6.]])

Os tensores do PyTorch realmente se assemelham a matrizes NumPy. Na verdade, eles têm mais de 200 funções comuns!

In [23]:
# extra code: list functions that appear both in NumPy and PyTorch
functions = lambda mod: set(f for f in dir(mod) if callable(getattr(mod, f)))
", ".join(sorted(functions(torch) & functions(np)))

'__getattr__, abs, absolute, acos, acosh, add, all, allclose, amax, amin, angle, any, arange, arccos, arccosh, arcsin, arcsinh, arctan, arctan2, arctanh, argmax, argmin, argsort, argwhere, asarray, asin, asinh, atan, atan2, atanh, atleast_1d, atleast_2d, atleast_3d, bincount, bitwise_and, bitwise_left_shift, bitwise_not, bitwise_or, bitwise_right_shift, bitwise_xor, broadcast_shapes, broadcast_to, can_cast, ceil, clip, column_stack, concat, concatenate, conj, copysign, corrcoef, cos, cosh, count_nonzero, cov, cross, cumprod, cumsum, deg2rad, diag, diagflat, diagonal, diff, divide, dot, dsplit, dstack, dtype, einsum, empty, empty_like, equal, exp, exp2, expm1, eye, finfo, fix, flip, fliplr, flipud, float_power, floor, floor_divide, fmax, fmin, fmod, frexp, from_dlpack, frombuffer, full, full_like, gcd, gradient, greater, greater_equal, heaviside, histogram, histogramdd, hsplit, hstack, hypot, i0, iinfo, imag, inner, isclose, isfinite, isin, isinf, isnan, isneginf, isposinf, isreal, kron

## 1.2 Hardware Acceleration

**Copiar tensores para a GPU** é simples no PyTorch, desde que sua máquina tenha GPU compatível e as bibliotecas necessárias estejam instaladas. No **Google Colab**, basta usar um **runtime com GPU** (Menu *Runtime* → *Change runtime type* → selecionar uma GPU, como **Nvidia T4**). Esse ambiente já vem com o PyTorch compilado com suporte a GPU, drivers e bibliotecas requeridas (por exemplo, **CUDA** e **cuDNN**).
Se preferir rodar localmente, instale **drivers** e **bibliotecas** apropriadas seguindo as instruções: [https://homl.info/install-p](https://homl.info/install-p).

O PyTorch tem excelente suporte a **GPUs Nvidia** e também a outros aceleradores:

* **Apple MPS**: aceleração em **Apple Silicon** (M1, M2, posteriores) e alguns **Intel Macs** compatíveis.
* **AMD**: **Instinct** e **Radeon** via **ROCm** (Linux) ou **DirectML** (Windows).
* **Intel**: **GPUs e CPUs** (Linux/Windows) via **oneAPI**.
* **Google TPUs**: integração via **`torch_xla`**.


In [24]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

device

'cuda'

Em um Colab GPU Runtime, o dispositivo será igual a "cuda". Agora, vamos criar um tensor nessa GPU. Para isso, uma opção é criar o tensor na CPU e copiá-lo para a GPU usando o método **to()**:

In [25]:
M = torch.tensor([[1., 2., 3.], [4., 5., 6.]])
M = M.to(device)
M.device

device(type='cuda', index=0)

Alternativamente, podemos criar o tensor diretamente na GPU usando o argumento do dispositivo:

In [26]:
M = torch.tensor([[1., 2., 3.], [4., 5., 6.]], device=device)

Quando o tensor estiver na GPU, podemos executar operações nele normalmente, e todas elas ocorrerão na GPU:

In [27]:
R = M @ M.T  # run some operations on the GPU
R

tensor([[14., 32.],
        [32., 77.]], device='cuda:0')

Quando um tensor é processado na **GPU**, o **resultado também permanece na GPU**. Isso permite **encadear várias operações** sem **copiar dados entre CPU e GPU**, evitando um gargalo comum de desempenho.

**Quanto a GPU acelera?** Depende do **modelo da GPU** (as mais caras podem ser **dezenas de vezes** mais rápidas) e do **throughput de dados**:

* **Modelos compute-heavy** (ex.: redes muito profundas): o **poder de cálculo** da GPU e a **quantidade de RAM** tendem a ser os fatores críticos.
* **Modelos rasos / datasets grandes**: o **envio contínuo de dados** para a GPU pode virar o **gargalo** principal.

O próximo trecho realiza um **teste comparando** a **multiplicação de matrizes** na **CPU vs GPU** para ilustrar essas diferenças de **tempo de execução** e **banda de dados**.


In [28]:
M = torch.rand((1000, 1000))  # on the CPU
M @ M.T  # warmup
%timeit M @ M.T

M = M.to(device)
M @ M.T  # warmup
%timeit M @ M.T

16.1 ms ± 2.17 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
549 µs ± 3.99 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


No teste, a **GPU (Nvidia T4 no Colab)** proporcionou um **ganho de ~26×** na multiplicação de matrizes. Com GPUs mais potentes, o **speedup** tende a ser ainda maior. Porém, **em matrizes pequenas** (ex.: `100 × 100`), o ganho cai para algo como **~2×**.
Isso ocorre porque **GPUs paralelizam tarefas grandes** (quebrando-as em muitas subtarefas para milhares de núcleos). **Tarefas pequenas** não geram paralelismo suficiente, reduzindo o benefício — e, em cenários com **muitas tarefas minúsculas**, **a CPU pode ser até mais rápida** devido ao overhead de orquestração na GPU.

Com a base de **tensores** e **execução em CPU/GPU** estabelecida, o próximo passo é explorar o **autograd** do PyTorch (auto-diferenciação).


## 1.3 Autograd

 PyTorch implementa **auto-diferenciação em modo reverso** (*autograd*), permitindo calcular **gradientes automaticamente**. A ideia é simples de usar: dado, por exemplo, $f(x) = x^2$, o cálculo diferencial diz que $f'(x)=2x$. Avaliando em $f(5)$, obtemos $f(5)=25$ e $f'(5)=10$. O próximo trecho verifica esses valores com o **autograd** do PyTorch.


In [29]:
x = torch.tensor(5.0, requires_grad=True)
f = x ** 2
f

tensor(25., grad_fn=<PowBackward0>)

In [30]:
f.backward()
x.grad

tensor(10.)

Obtivemos os valores corretos: **$f=25$** e **(x.\texttt{grad}=10)**. A chamada `backward()` calculou automaticamente $f'(x)$ no ponto (x=5.0). Eis a lógica, linha a linha:

* **Definição de variável com gradiente**
  Criamos `x = 5.0` com `requires_grad=True`. Assim, o PyTorch **rastreia todas as operações** envolvendo `x`, construindo o **grafo de computação** necessário para executar o **backpropagation**. Nesse grafo, `x` é um **nó-folha**.

* **Cálculo de $f = x ** 2$**
  O resultado é `25.0`. Além do valor, `f` carrega o atributo **`grad_fn`** (ex.: `PowBackward0`), que **representa a operação** que gerou `f` e **informa como retropropagar** os gradientes por essa operação. É assim que o PyTorch **mantém o grafo**.

* **Retropropagação**
  `f.backward()` **propaga gradientes** a partir de `f` **até os nós-folha** (aqui, apenas `x`).

* **Leitura do gradiente**
  `x.grad` contém a **derivada de $f$ em relação a (x)**, computada no backprop (no exemplo, **10**).

### Grafos dinâmicos (define-by-run)

O PyTorch **cria o grafo on-the-fly** a cada *forward pass*, conforme as operações são executadas, o que suporta **modelos dinâmicos** com **loops e condicionais**.

### Passo de gradiente (gradient descent) com `torch.no_grad()`

Após obter gradientes, normalmente fazemos **descida do gradiente**, subtraindo uma fração do gradiente das variáveis do modelo. No exemplo de $f(x)=x^2$, isso **empurra (x) em direção a 0** (o minimizador).

> Importante: **desative o rastreamento de gradiente** durante a atualização (por exemplo, usando `with torch.no_grad():`), pois **não queremos registrar** a própria atualização no grafo — e operações *in-place* em variáveis rastreadas podem **levantar exceção**.


In [31]:
learning_rate = 0.1
with torch.no_grad():
    x -= learning_rate * x.grad  # gradient descent step

In [32]:
x

tensor(4., requires_grad=True)

No passo de **descida do gradiente**, (x) é decrementado por $0{,}1 \times 10{,}0 = 1{,}0$, indo de **5,0** para **4,0**.

Outra forma de **evitar cálculo de gradientes** é usar **`detach()`**: ele cria um **novo tensor desacoplado do grafo** (`requires_grad=False`), **apontando para os mesmos dados em memória**. Assim, você pode **atualizar esse tensor desacoplado** sem registrar a operação no grafo de computação.

In [33]:
x_detached = x.detach()
x_detached -= learning_rate * x.grad

Como $x_{\text{detached}}$ e $x$ **compartilham a mesma memória**, **alterar** $x_{\text{detached}}$ também **altera** $x$.

* **`detach()`**: útil quando você precisa **executar computações sem afetar os gradientes** (ex.: avaliação, logging) ou quando deseja **controle fino** sobre **quais operações** entram no cálculo de gradientes. Cria um **tensor desacoplado do grafo** ( `requires_grad=False` ), apontando para os **mesmos dados**.
* **`no_grad()`**: geralmente **preferido** para **inferência** ou durante o **passo de descida do gradiente**, pois fornece um **contexto** conveniente que **desativa o rastreamento** de gradientes para tudo que estiver dentro do bloco.

**Antes de repetir o ciclo** *(forward → backward → atualização)*, é **essencial zerar os gradientes** de **todos os parâmetros** do modelo, pois o PyTorch **acumula** gradientes por padrão. Não é necessário `no_grad()` para isso, já que os tensores de gradiente têm `requires_grad=False`.


In [34]:
x.grad.zero_()

tensor(0.)

Juntando tudo, o ciclo de treinamento fica assim:

In [35]:
learning_rate = 0.1
x = torch.tensor(5.0, requires_grad=True)
for iteration in range(100):
    f = x ** 2  # forward pass
    f.backward()  # backward pass
    with torch.no_grad():
        x -= learning_rate * x.grad  # gradient descent step
    x.grad.zero_()  # reset the gradients

In [36]:
x

tensor(1.0185e-09, requires_grad=True)

Operações **in-place** podem **economizar memória** e **evitar cópias**, mas **nem sempre** combinam bem com o **autograd**:

1. **Proibição em nós-folha**
   Você **não pode** aplicar uma operação *in-place* em um **nó-folha** (tensor com `requires_grad=True`). O PyTorch não saberia **onde armazenar** as informações do grafo. Exemplos que geram `RuntimeError`:

   * `x.cos_()`
   * `x += 1`

2. **Risco de sobrescrever valores necessários ao backward**
   Mesmo fora de nós-folha, operações *in-place* podem **apagar valores intermediários** que o autograd precisa para calcular gradientes, quebrando o **backpropagation**.

**Exemplo a seguir no código**: calcular $z(t) = \exp(t) + 1$ em $t=2$ e tentar obter os gradientes — veremos como certas escolhas *in-place* podem levar a erros durante o `backward()`.


In [ ]:
#Novo

t = torch.tensor(2.0, requires_grad=True)
z = t.exp()  # this is an intermediate result
z += 1  # this is an in-place operation
z.backward()  # ⚠️ RuntimeError!

O valor de $z$ foi calculado corretamente, mas o `backward()` lançou `RuntimeError: one of the variables needed for gradient computation has been modified by an in-place operation`. Isso ocorreu porque o resultado intermediário $z = t.\exp()$ foi **sobrescrito** por `z += 1`. No *backward*, ao chegar na **exp**, o autograd já não tinha o valor necessário para calcular o gradiente.

**Correção simples:** troque `z += 1` por `z = z + 1`. Visualmente é parecido, mas **não é in-place**: cria-se **um novo tensor**, preservando o original no **grafo de computação**.

---

### Por que às vezes funciona? (ex.: com `cos()`)

Se você trocar `exp()` por `cos()` no exemplo, o gradiente **funciona**. O motivo é **como cada operação é implementada** e **o que ela guarda** para o *backward*:

* **Operações que salvam a *saída*** (não modifique a **saída** *in-place* antes do *backward*):
  `exp()`, `relu()`, `rsqrt()`, `sigmoid()`, `sqrt()`, `tan()`, `tanh()`.

* **Operações que salvam a *entrada*** (podem tolerar modificar a **saída**, mas **não** modifique a **entrada** *in-place* antes do *backward*):
  `abs()`, `cos()`, `log()`, `sin()`, `square()`, `var()`.

* **Operações que salvam *entrada e saída*** (não modifique **nenhuma** delas *in-place*):
  `max()`, `min()`, `norm()`, `prod()`, `sgn()`, `std()`.

* **Operações que não salvam nem entrada nem saída** (seguro modificar *in-place*):
  `ceil()`, `floor()`, `mean()`, `round()`, `sum()`.

> **Regra prática:** quando usar operações *in-place* para economizar memória, verifique **o que a operação salva** para o *backward*. Se você alterar *in-place* algo que o autograd precisa (entrada, saída ou ambos), o gradiente **quebrará**.


OK, vamos voltar um pouco. Já discutimos todos os fundamentos do PyTorch: como criar tensores e usá-los para realizar todos os tipos de cálculos, como acelerar os cálculos com uma GPU e como usar o Autograd para calcular gradientes para gradiente descendente. Ótimo! Agora, vamos aplicar o que aprendemos até agora, construindo e treinando um modelo de regressão linear simples com o PyTorch.

# 2. Implementing Linear Regression

Começaremos implementando a regressão linear usando tensores e autograd diretamente, depois simplificaremos o código usando a API de alto nível do PyTorch e também adicionaremos suporte à GPU.

## 2.1 Linear Regression Using Tensors & Autograd

Usaremos o **California Housing** (como no Cap. 9). Suponha que os dados já foram baixados com `sklearn.datasets.fetch_california_housing()` e divididos com `train_test_split()` em: `X_train`, `y_train`, `X_valid`, `y_valid`, `X_test`, `y_test`.

Agora vamos **converter tudo para tensores** e **normalizar** usando **operações de tensor** (em vez de `StandardScaler`), para praticar PyTorch:

* **Estatísticas no treino**: calcule **média** $\mu_{\text{train}}$ e **desvio-padrão** $\sigma_{\text{train}}$ apenas em `X_train`.
* **Padronização**: aplique em todos os *splits* a transformação
  $$\tilde{X}=\frac{X-\mu_{\text{train}}}{\sigma_{\text{train}}}$$

Isso garante padronização consistente e evita **vazamento de informação**. O próximo trecho mostra a **conversão para tensores** e a **normalização** passo a passo.

In [37]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

In [38]:
X_train = torch.FloatTensor(X_train)
X_valid = torch.FloatTensor(X_valid)
X_test = torch.FloatTensor(X_test)
means = X_train.mean(dim=0, keepdims=True)
stds = X_train.std(dim=0, keepdims=True)
X_train = (X_train - means) / stds
X_valid = (X_valid - means) / stds
X_test = (X_test - means) / stds

Também precisamos **converter os alvos para tensores**. Como as **predições** serão **vetores-coluna** (matrizes com **uma única coluna**), os **alvos** devem ter o **mesmo formato**. Porém, no NumPy os alvos vêm **unidimensionais**; portanto, é necessário **reestruturar** para **adicionar uma 2ª dimensão de tamanho 1**, formando vetores-coluna (isto é, shape $[N,,1]$).

> Em resumo: converta `y_*` para tensores e **reshape** para $[N,,1]$ para alinhar com as **saídas do modelo**.


In [39]:
y_train = torch.FloatTensor(y_train).view(-1, 1)
y_valid = torch.FloatTensor(y_valid).view(-1, 1)
y_test = torch.FloatTensor(y_test).view(-1, 1)

Agora que os dados estão prontos, vamos criar os parâmetros do nosso modelo de regressão linear:

In [40]:
torch.manual_seed(42)
n_features = X_train.shape[1]  # there are 8 input features
w = torch.randn((n_features, 1), requires_grad=True)
b = torch.tensor(0., requires_grad=True)

Definimos um vetor de **pesos** $w$ (vetor-coluna com **um peso por atributo de entrada**, aqui $8$) e um **viés** escalar $b$. Os **pesos são inicializados aleatoriamente** e o **viés em zero**. Embora, neste caso simples, também pudéssemos zerar $w$, é boa prática **inicializar pesos aleatoriamente** para **quebrar a simetria** entre unidades — princípio crucial em **redes neurais** (Cap. 9). Adotar esse hábito desde já facilita a transição para modelos mais profundos.

Em seguida, vamos treinar nosso modelo, de forma muito semelhante à que fizemos no Capítulo 4, exceto que usaremos o autodiff para calcular os gradientes em vez de usar uma equação de forma fechada. Por enquanto, usaremos a descida do gradiente em lote (BGD), utilizando o conjunto de treinamento completo em cada etapa:

### Laço de treino: passo a passo (regressão linear)

* **Taxa de aprendizado (`learning_rate`)**
  Definimos o hiperparâmetro de **taxa de aprendizado**; experimente valores para equilibrar **convergência** e **precisão**.

* **Épocas (20)**
  Executamos **20 épocas**. Poderíamos aplicar **early stopping** (Cap. 4), mas aqui mantemos simples.

* **Forward pass**
  Calculamos as **predições** $y_{\text{pred}}$ e a **loss MSE**:
  $$\mathrm{MSE}(y,\hat y)=\frac{1}{N}\sum_{i=1}^{N}\bigl(y_i-\hat y_i\bigr)^2.$$

* **Autograd**
  `loss.backward()` calcula **gradientes** da loss em relação a **todos os parâmetros**.

* **Passo de descida do gradiente**
  Usamos `b.grad` e `w.grad` para **atualizar** os parâmetros **dentro de** `with torch.no_grad():`.

* **Zerar gradientes (essencial!)**
  Após atualizar, **zeramos os gradientes** para não acumulá-los na próxima iteração.

* **Logging**
  Imprimimos **nº da época** e a **loss**; `item()` extrai o valor escalar.

> Regra prática de formatação: **código** → `backticks`; **fórmulas** → `$...$`; se precisar de sublinhado em modo math, use `\_`.



In [42]:
learning_rate = 0.4
n_epochs = 20
for epoch in range(n_epochs):
    y_pred = X_train @ w + b
    loss = ((y_pred - y_train) ** 2).mean()
    loss.backward()
    with torch.no_grad():
        b -= learning_rate * b.grad
        w -= learning_rate * w.grad
        b.grad.zero_()
        w.grad.zero_()
    print(f"Epoch {epoch + 1}/{n_epochs}, Loss: {loss.item()}")

Epoch 1/20, Loss: 16.158456802368164
Epoch 2/20, Loss: 4.8793745040893555
Epoch 3/20, Loss: 2.255225419998169
Epoch 4/20, Loss: 1.3307634592056274
Epoch 5/20, Loss: 0.9680691957473755
Epoch 6/20, Loss: 0.8142675757408142
Epoch 7/20, Loss: 0.7417045831680298
Epoch 8/20, Loss: 0.7020701169967651
Epoch 9/20, Loss: 0.6765918731689453
Epoch 10/20, Loss: 0.6577965021133423
Epoch 11/20, Loss: 0.6426151990890503
Epoch 12/20, Loss: 0.6297222971916199
Epoch 13/20, Loss: 0.6184942126274109
Epoch 14/20, Loss: 0.6085968613624573
Epoch 15/20, Loss: 0.5998216867446899
Epoch 16/20, Loss: 0.592018723487854
Epoch 17/20, Loss: 0.5850691795349121
Epoch 18/20, Loss: 0.578873336315155
Epoch 19/20, Loss: 0.573345422744751
Epoch 20/20, Loss: 0.5684100389480591


Parabéns, você acabou de treinar seu primeiro modelo usando o PyTorch! Agora você pode usar o modelo para fazer previsões para alguns novos dados X_new (que devem ser representados como um tensor do PyTorch). Por exemplo, vamos fazer previsões para as três primeiras instâncias do conjunto de teste:

In [43]:
X_new = X_test[:3]  # pretend these are new instances
with torch.no_grad():
    y_pred = X_new @ w + b  # use the trained parameters to make predictions

In [44]:
y_pred

tensor([[0.8916],
        [1.6480],
        [2.6577]])

Implementar a regressão linear usando a API de baixo nível do PyTorch não foi tão difícil, mas usar essa abordagem para modelos mais complexos seria muito complicado e trabalhoso. Portanto, o PyTorch oferece uma API de alto nível para simplificar tudo isso. Vamos reescrever nosso modelo usando essa API de alto nível.

## 2.2 Linear Regression Using PyTorch's High-Level API

PyTorch fornece uma implementação de regressão linear na classe `torch.nn.Linear`, então vamos usá-la
> **nn = *neural network(s)*** (“rede(s) neural(is)”).

In [45]:
import torch.nn as nn

torch.manual_seed(42)  # to get reproducible results
model = nn.Linear(in_features=n_features, out_features=1)

`nn.Linear` (atalho de `torch.nn.Linear`) é um **módulo** do PyTorch; todo módulo herda de `nn.Module`. Para **regressão linear simples**, **um único** `nn.Linear` já basta. Em redes neurais reais, você **empilha vários módulos** — pense neles como **“LEGOs matemáticos”**.

**Parâmetros internos do `nn.Linear`:**

* **Viés**: vetor de **bias** com **um termo por neurônio**.
* **Pesos**: **matriz de pesos** com **uma linha por neurônio** e **uma coluna por dimensão de entrada** (ou seja, é a **transposta** da matriz/vetor de pesos usada anteriormente no capítulo e na equação abaixo).

No nosso caso, com `out_features=1`, o modelo tem **um único neurônio**:

* O vetor de **bias** tem **um único termo**.
* A **matriz de pesos** tem **uma única linha**.

Esses parâmetros ficam acessíveis **diretamente** como **atributos** do módulo:

* `linear.weight`  → matriz de pesos $W$
* `linear.bias`    → viés $b$

> Intuição: o módulo implementa $,y = XW^\top + b,$ (com $W$ armazenado como **linha** quando há um neurônio de saída), mantendo os parâmetros **dentro do módulo** para integração com o **autograd** e os **otimizadores**.


<div align="center" style="margin-top:20px; margin-bottom:20px;">
  
![pt](Aula_Imagens\eq_Percep.png)

</div>


In [46]:
model.bias

Parameter containing:
tensor([0.3117], requires_grad=True)

In [47]:
model.weight

Parameter containing:
tensor([[ 0.2703,  0.2935, -0.0828,  0.3248, -0.0775,  0.0713, -0.1721,  0.2076]],
       requires_grad=True)

### Parâmetros em `nn.Linear`: inicialização, `Parameter` e iteração

* **Inicialização aleatória + reprodutibilidade**
  Os **dois parâmetros** do módulo (`weight` e `bias`) são **inicializados aleatoriamente** por padrão — por isso utilizamos `manual_seed()` para obter **resultados reprodutíveis**.

* **`torch.nn.Parameter` vs tensor comum**
  Esses parâmetros são instâncias de **`torch.nn.Parameter`**, que **herda** de `tensor.Tensor`.
  ➤ Consequência: você pode **usá-los como tensores normais** (mesmas operações), **mas** o PyTorch os **registra automaticamente** como **parâmetros treináveis** do módulo (para autograd e otimizadores).
  **Diferença central:** um **`Parameter`** é um **tensor marcado** para ser **rastreado como parâmetro do modelo**; já um **tensor comum**, mesmo com `requires_grad=True`, **não** é listado como parâmetro do módulo.

* **`module.parameters()` (recursivo)**
  O método **`parameters()`** retorna um **iterador** sobre **todos os atributos do tipo `Parameter`** do módulo **e de seus submódulos (recursivamente)**.
  Não retorna **tensores comuns**, mesmo que tenham `requires_grad=True`.

> Em resumo: **`Parameter` = tensor treinável do módulo** (aparece em `parameters()` e nos otimizadores); **tensor comum** pode participar do grafo, mas **não** é tratado como **parâmetro do modelo**.


In [48]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[ 0.2703,  0.2935, -0.0828,  0.3248, -0.0775,  0.0713, -0.1721,  0.2076]],
       requires_grad=True)
Parameter containing:
tensor([0.3117], requires_grad=True)


* **Parâmetros nomeados**
  Além de `parameters()`, há **`named_parameters()`**, que retorna um **iterador de pares (nome, parâmetro)**. Útil para **inspeção**, **debug** e **logs** (ex.: imprimir gradientes por nome).

* **Chamando o módulo como função**
  Um **módulo** (`nn.Module`) pode ser **chamado como função**: internamente ele executa o método `forward`.
  Exemplo: passar as **duas primeiras instâncias** do *training set* ao modelo para obter **predições** iniciais.

  > Como os **parâmetros ainda estão aleatórios**, as **predições serão ruins** — o comportamento esperado **antes do treino**.


In [49]:
model(X_train[:2])

tensor([[-0.4718],
        [ 0.1131]], grad_fn=<AddmmBackward0>)

Ao usar um **módulo como função**, o PyTorch chama internamente seu **`forward()`**. No caso de `nn.Linear`, ele computa:
[
X ;@; W^\top ;+; b
]
onde $X$ é a entrada, $W$ é `self.weight` e $b$ é `self.bias` — **exatamente** o que precisamos para **regressão linear**.

O tensor de saída traz o atributo **`grad_fn`**, indicando que o **autograd** rastreou o **grafo de computação** durante as predições.

**Próximo passo:** com o modelo definido, precisamos criar um **otimizador** (para **atualizar os parâmetros**) e escolher uma **função de perda** (loss).


In [50]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
mse = nn.MSELoss()

O PyTorch oferece **diversos otimizadores** (detalhes no próximo capítulo). Aqui usamos o **SGD** (*stochastic gradient descent*), que atende a **SGD puro**, **mini-batch GD** e **batch GD**. Para inicializá-lo, passamos os **parâmetros do modelo** (por exemplo, `model.parameters()`) e a **taxa de aprendizado** (`lr`).

Para a **função de perda**, instanciamos **`nn.MSELoss`**. Ela também é um **módulo**, então pode ser usada **como função**: recebe **predições** e **alvos** e calcula a **MSE**,

  $$\mathrm{MSE}(y,\hat y)=\frac{1}{N}\sum_{i=1}^{N}\bigl(y_i-\hat y_i\bigr)^2.$$
  
O submódulo `nn` inclui **muitas outras perdas** e utilidades de redes neurais.

**A seguir:** escreveremos uma **função de treino** compacta para executar o ciclo *forward → loss → backward → update*.

In [51]:
def train_bgd(model, optimizer, criterion, X_train, y_train, n_epochs):
    for epoch in range(n_epochs):
        y_pred = model(X_train)
        loss = criterion(y_pred, y_train)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        print(f"Epoch {epoch + 1}/{n_epochs}, Loss: {loss.item()}")

### Comparando os laços de treino: baixo nível vs. alto nível (`nn` + `optim`)

O novo laço é **quase idêntico** ao anterior, porém agora usamos **abstrações de nível mais alto** (módulos `nn` e otimizadores `optim`) em vez de manipular diretamente **tensores** e **autograd**. Pontos-chave:

* **Critério (loss function “objeto”)**
  Em PyTorch, é comum chamar o **objeto** de função de perda de **criterion** (para distinguir do **valor da loss** calculado a cada iteração).
  Aqui, o **criterion** é uma instância de `nn.MSELoss`.

* **`optimizer.step()`**
  Substitui as duas linhas manuais que **atualizavam** $b$ e $w$ no código anterior (descida do gradiente).

* **`optimizer.zero_grad()`**
  Substitui as duas linhas que **zeravam** `b.grad` e `w.grad`.
  Observação: **não é necessário** usar `with torch.no_grad():` aqui — o **otimizador** já trata corretamente o **rastro de gradientes** internamente em `step()` e `zero_grad()`.

**Próximo passo:** chamar a **função de treino** para ajustar o modelo e observar a **queda da loss** ao longo das épocas.


In [52]:
train_bgd(model, optimizer, mse, X_train, y_train, n_epochs)

Epoch 1/20, Loss: 4.3378496170043945
Epoch 2/20, Loss: 0.7802939414978027
Epoch 3/20, Loss: 0.6253842115402222
Epoch 4/20, Loss: 0.6060433983802795
Epoch 5/20, Loss: 0.5956299304962158
Epoch 6/20, Loss: 0.587356686592102
Epoch 7/20, Loss: 0.5802990794181824
Epoch 8/20, Loss: 0.5741382837295532
Epoch 9/20, Loss: 0.5687101483345032
Epoch 10/20, Loss: 0.5639079809188843
Epoch 11/20, Loss: 0.5596511363983154
Epoch 12/20, Loss: 0.5558737516403198
Epoch 13/20, Loss: 0.5525194406509399
Epoch 14/20, Loss: 0.5495392084121704
Epoch 15/20, Loss: 0.5468900203704834
Epoch 16/20, Loss: 0.544533908367157
Epoch 17/20, Loss: 0.5424376726150513
Epoch 18/20, Loss: 0.5405716300010681
Epoch 19/20, Loss: 0.5389097332954407
Epoch 20/20, Loss: 0.5374288558959961


Tudo bem; o modelo foi treinado e agora você pode usá-lo para fazer previsões simplesmente chamando-o como uma função (de preferência dentro de um contexto no_grad(), como vimos anteriormente):

In [53]:
X_new = X_test[:3]  # pretend these are new instances
with torch.no_grad():
    y_pred = model(X_new)  # use the trained model to make predictions

y_pred

tensor([[0.8061],
        [1.7116],
        [2.6973]])

As **predições** ficam **semelhantes**, mas **não idênticas** às do modelo anterior porque o `nn.Linear` **inicializa os parâmetros de forma diferente**: usa uma **distribuição uniforme** em um **intervalo específico** tanto para os **pesos** quanto para o **viés** (detalhes sobre inicialização serão tratados no **Cap. 11**).

Com essa **API de alto nível** dominada, estamos prontos para ir além da **regressão linear** e construir um **perceptron multicamadas (MLP)**.


![NCIA2](NCIA_Images\end.png)